In [ ]:
from cobra.io import read_sbml_model
mod = read_sbml_model('C:/Users/prins/git/Human-GEM/model/Human-GEM.xml') 
import os
folder = "C:/Users/prins/git/Human1_RPE-PR/cs_mods"
files = [f for f in os.listdir(folder) if '.xml'in f if os.path.isfile(os.path.join(folder, f))]
models = {f:read_sbml_model(folder + '/' + f ) for f in files}
model_n = [f.split('_')[0] + f.split('_')[1] for f in files] 

In [ ]:
model_n2 = ['Schwan (1)', 'Schwan (2)', 'Melanocytes', 'Endothelial','Muscle','Fibroblasts','RPE','B cells','T + NK cells',\
            'Monocytes + Macrophages','Mast','Retina','RPE + Choroid','PR rod','Amacrine','Bipolar',\
           'PR cone', 'Horizontal', 'Müller','Ganglion']
model_n2
model_n3 = list(range(1,21))

In [ ]:
mod = read_sbml_model('C:/Users/prins/git/Human-GEM/model/Human-GEM.xml') 
mod

In [ ]:
mod_RPE = models['model_06_MeanExpression_RPE__VoigtEtAl2019_.xml'].copy()
mod_PR = models['model_13_MeanExpression_RodPhotoreceptors__LiangEtAl_2019_.xml'].copy()

In [ ]:
def remove_compartment(model, compartment):
# remove compartment from model
    rxns_n = [r.id for r in model.reactions if compartment in r.compartments]
    mets_n = [m for m in model.metabolites if compartment in m.compartment]
    model.remove_reactions(rxns_n)
    model.remove_metabolites(mets_n)
    return model

# remove nucleus from PR model
mod_PR = remove_compartment(mod_PR, 'n')

In [ ]:
def add_id_suffix(model, suffix):
    for r in model.reactions:
        r.id = r.id + suffix
    for m in model.metabolites:
        m.id = m.id + suffix
        m.compartment = m.compartment + suffix
    return model

# add '_PR' to PR model IDs and '_RPE' to RPE model IDs
mod_PR = add_id_suffix(mod_PR, '_PR')
mod_RPE = add_id_suffix(mod_RPE, '_RPE')

In [ ]:
# add RPE-PR interface reactions (rxn_RPE-PR = RPE <--> RPE-PR interface)
for rxns in  [r for r in mod_RPE.reactions if 'e_RPE' in r.reaction]:     
    rxn = rxns.copy()
    rxn.id = rxn.id.replace('_RPE','_RPE-PR')
    for p in rxn.products:
        if 'e_RPE' in p.id:
            p.id = p.id.replace('e_RPE','e_RPE-PR')
    for r in rxn.reactants:
        if 'e_RPE' in r.id:
            r.id = r.id.replace('e_RPE','e_RPE-PR')
    mod_RPE.add_reactions([rxn])
    
# add RPE-PR interface reactions (rxn_PR-RPE = PR <--> RPE-PR interface)
for rxns in  [r for r in mod_PR.reactions if 'e_PR' in r.reaction]:     
    rxn.id = rxn.id.replace('_PR','_PR-RPE')
    for p in rxn.products:
        if 'e_PR' in p.id:
            p.id = p.id.replace('e_PR','e_RPE-PR')
    for r in rxn.reactants:
        if 'e_PR' in r.id:
            r.id = r.id.replace('e_PR','e_RPE-PR')
    mod_PR.add_reactions([rxn])

In [ ]:
 [r.build_reaction_string(use_metabolite_names = False) for r in mod_RPE.reactions if '_RPE-PR' in r.reaction]

In [ ]:

for m in [m for m in mod_PR.metabolites if m.compartment=='e_PR']:
    m.compartment = 'e_RPE-PR'

In [ ]:
# make RPE-PR interface
for m in [m for m in mod_PR.metabolites if m.compartment=='e_PR']:
    m.compartment = 'e_RPE-PR'
    
# for r in [r for r in mod_RPE.reactions if 'e_RPE' in r.reaction]:
#    r_RPE_PE = r.copy()
#    for p in r.products:
#        p.id = p.id + '-PR'
#    for p in r.reactants:
#        p.id = p.id + '-PR'

In [ ]:
r = [r for r in mod_RPE.reactions if 'e_RPE' in r.reaction][0].copy()
for p in r.products:
    p.id = p.id + '-PR'
    print(p.id)

In [ ]:
mod.reactions.get_by_id('MAR01862')

In [ ]:
if 'vmhreaction' in mod.reactions[0].annotation.keys(): 
    print(mod.reactions[0].annotation['vmhreaction'])

In [ ]:
len(([m for m in mod.metabolites]))

In [ ]:
vmh_met = [str(m.annotation['vmhmetabolite']) + '_' + m.compartment if 'vmhmetabolite' in m.annotation.keys() else '' for m in mod.metabolites]

In [ ]:
mod.metabolites[0].name

In [ ]:
mod.metabolites[0].annotation

In [ ]:
import pandas as pd
subsystems = list(set([r.subsystem for r in mod.reactions]))
df = pd.DataFrame(index=subsystems)
for i in range(len(files)):
    count = [[rxn.subsystem for rxn in models[files[i]].reactions].count(s) for s in subsystems]
    df[model_n2[i]] = count 
df

In [ ]:
len(atp_rxns)

In [ ]:
atp_rxns = [list(m.reactions) for m in mod.metabolites if 'ATP' == m.name] 
atp_rxns = list(set([i for atp_rxns_i in atp_rxns for i in atp_rxns_i])) # unique 
df_atp_rxns = pd.DataFrame([[r.id,\
              r.annotation['vmhreaction'] if 'vmhreaction' in r.annotation.keys() else '', \
              r.name, \
              r.build_reaction_string(use_metabolite_names = True)]\
              for r in atp_rxns])
df_atp_rxns

In [ ]:
import pandas as pd
subsystems = list(set([r.subsystem for r in mod.reactions]))
df_h1 = pd.DataFrame(index=subsystems)
count = [[rxn.subsystem for rxn in mod.reactions].count(s) for s in subsystems]
df_h1['Human1'] = count 
df['Human1']=df_h1
df

In [ ]:
df['Human1']=df_h1

df2=df[(df == 0).any(axis=1)] # select rows and columns containing zeros
idx_row = df2[df2==0].count(axis=1).sort_values(ascending=False).index.to_list()
idx_col = df2[df2==0].count(axis=0).sort_values(ascending=False).index.to_list()
df2 = df2.reindex(idx_col, axis=1).reindex(idx_row,axis=0)

def style_negative(v, props=''):
    return props if v == 0 else None
df2 = df2[(df == 0).any(axis=1)].style.applymap(style_negative, props='background-color:gray;').\
set_table_styles([dict(selector="th", props=[('width', '10px')]),
              dict(selector="th.col_heading",
                   props=[("writing-mode", "vertical-rl"),
                          ('transform', 'rotateZ(180deg)'), 
                          ('height', '170px'),
                          ('vertical-align', 'top')]),
              dict(selector="th.row_heading",
                   props=[('width', '450px'),])])
df2

In [ ]:
df_norm = df.drop(['Human1'], axis=1).divide(df['Human1'].values, axis='rows')
idx = df_norm.mean(axis=1).sort_values(ascending=False).index.to_list()
df_norm['Human1 (number of reactions)']=df['Human1']
df_norm=df_norm.reindex(idx,axis=0)
df_norm.to_clipboard(excel=True, sep=None)
df_norm

In [ ]:
[r.id for r in mod.reactions if r.subsystem == 'Glycolysis / Gluconeogenesis']
pd.DataFrame([[r.id,r.name,r.build_reaction_string(use_metabolite_names = True)] for r in mod.reactions if r.subsystem == 'Glycolysis / Gluconeogenesis'], columns=['id','name','reaction']).to_clipboard(excel=True, sep=None)

In [ ]:
rxns_mod = [r.id for r in mod.reactions if r.subsystem == 'Glycolysis / Gluconeogenesis']
l_bin_gly_rxns = [[1 if r in rxns_i else 0 for r in rxns_mod] for rxns_i in [[r.id for r in models[f].reactions if r.subsystem == 'Glycolysis / Gluconeogenesis'] for f in files]]
df_gly_rxns = pd.DataFrame([[r.id,r.name] for r in mod.reactions if r.subsystem == 'Glycolysis / Gluconeogenesis'], columns=['id','name'])
for i in range(len(model_n2)):
    df_gly_rxns[model_n2[i]]=l_bin_gly_rxns[i]

idx_row = df_gly_rxns[df_gly_rxns==0].count(axis=1).sort_values(ascending=True).index.to_list()
idx_col = df_gly_rxns[df_gly_rxns==0].count(axis=0).sort_values(ascending=True).index.to_list()
df_gly_rxns = df_gly_rxns.reindex(idx_col, axis=1).reindex(idx_row,axis=0)
df_gly_rxns.to_clipboard(excel=True, sep=None)
df_gly_rxns